In [30]:
import pandas as pd

In [31]:
train_path = '../Data/widsdatathon2024-university/train.csv'
test_path = '../Data/widsdatathon2024-university/test.csv'

output_path = '../Data/train_test_tt.csv'

In [32]:
train_df = pd.read_csv(train_path)
train_df['allocated_set'] = 'train'
test_df = pd.read_csv(test_path)
test_df['allocated_set'] = 'test'

df = pd.concat([train_df,test_df],
              axis=0).reset_index(drop=True)

In [33]:
df.describe()

patient_id  patient_zip3   patient_age           bmi  \
count   39519.000000  39519.000000  39519.000000  12482.000000   
mean   550399.374174    520.365116     54.424049     29.229523   
std    260773.401143    292.656985     11.217219      5.662184   
min    100051.000000      6.000000     18.000000     14.000000   
25%    323049.000000    292.000000     47.000000     24.960000   
50%    552167.000000    488.000000     55.000000     28.660000   
75%    776035.000000    782.000000     62.000000     33.000000   
max    999996.000000    999.000000     91.000000     97.000000   

       breast_cancer_diagnosis_year    population       density    age_median  \
count                  39519.000000  39519.000000  39519.000000  39519.000000   
mean                    2016.486424  19887.925833   1656.668336     40.689346   
std                        1.057922  13489.353665   3642.158636      4.012954   
min                     2015.000000     11.000000      0.821739     20.600000   
25%                     2016.000000   8917.000000    167.167647     37.734426   
50%                     2016.000000  18259.000000    627.904167     40.768605   
75%                     2017.000000  27537.000000   1555.107692     43.212963   
max                     2018.000000  71374.000000  29851.685710     73.500000   

       age_under_10  age_10_to_19  ...    race_other  race_multiple  \
count  39519.000000  39519.000000  ...  39519.000000   39519.000000   
mean      11.118332     12.886369  ...      5.346427       6.452429   
std        1.532715      1.881145  ...      5.956250       3.518786   
min        0.000000      0.000000  ...      0.000000       0.000000   
25%       10.149254     11.792453  ...      1.307353       3.892593   
50%       11.022034     12.923944  ...      3.355556       5.578689   
75%       12.194737     13.930435  ...      7.630000       8.611905   
max       17.675000     35.300000  ...     33.188889      26.429870   

           hispanic      disabled       poverty  limited_english  \
count  39519.000000  39518.000000  39514.000000     39514.000000   
mean      17.788920     13.437071     13.318639         4.614742   
std       17.493325      3.798747      5.760003         6.687422   
min        0.000000      4.600000      3.433333         0.000000   
25%        4.608824     10.270492      9.329577         0.972414   
50%       11.588889     13.085185     12.122857         2.605882   
75%       24.485185     15.657143     16.408333         5.958333   
max       99.104348     35.155556     48.931818        74.067391   

       commute_time  health_uninsured       veteran  treatment_pd  
count  39518.000000      39518.000000  39519.000000  27674.000000  
mean      28.100588          8.552149      7.072006    134.218472  
std        4.918609          4.317622      3.102473    187.759654  
min       12.460784          1.165385      1.200000      0.000000  
25%       24.986420          5.472464      4.956250     36.000000  
50%       27.828125          7.382500      6.882222     70.000000  
75%       30.925000         10.676316      8.640000    133.000000  
max       48.020000         29.565000     65.700000   1455.000000  

[8 rows x 70 columns]

In [34]:
df.columns

Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'patient_age', 'patient_gender', 'bmi',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'breast_cancer_diagnosis_year', 'metastatic_cancer_diagnosis_code',
       'metastatic_first_treatment', 'metastatic_first_treatment_type',
       'metastatic_first_novel_treatment',
       'metastatic_first_novel_treatment_type', 'region', 'division',
       'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19',
       'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
       'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married',
       'widowed', 'family_size', 'family_dual_income',
       'income_household_median', 'income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_5

# split breast cancer diagnosis into sides and quadrants

In [38]:
map_cancer_desc = {
    'Malignant neoplasm of ovrlp sites of right female breast': {'side':'R', 'quadrant':'overlap'},
    'Malig neoplasm of upper-outer quadrant of left female breast': {'side':'L', 'quadrant':'UO'},
    'Malig neoplasm of upper-inner quadrant of left female breast': {'side':'L', 'quadrant':'UI'},
    'Malignant neoplasm of breast (female), unspecified': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malig neoplm of lower-inner quadrant of right female breast': {'side':'R', 'quadrant':'LI'},
    'Malignant neoplasm of upper-inner quadrant of female breast': {'side':'unspecified', 'quadrant':'UI'},
    'Malignant neoplasm of central portion of left female breast': {'side':'L', 'quadrant':'central'},
    'Malignant neoplasm of unspecified site of left female breast': {'side':'L', 'quadrant':'unspecified'},
    'Malig neoplm of upper-inner quadrant of right female breast': {'side':'R', 'quadrant':'UI'},
    'Malignant neoplasm of unsp site of unspecified female breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malignant neoplasm of unsp site of right female breast': {'side':'R', 'quadrant':'unspecified'},
    'Malig neoplm of upper-outer quadrant of right female breast': {'side':'R', 'quadrant':'UO'},
    'Malig neoplasm of lower-inner quadrant of left female breast': {'side':'L', 'quadrant':'LI'},
    'Malignant neoplasm of nipple and areola, left female breast': {'side':'L', 'quadrant':'nipple'},
    'Malignant neoplasm of lower-outer quadrant of female breast': {'side':'unspecified', 'quadrant':'LO'},
    'Malignant neoplasm of other specified sites of female breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malig neoplm of lower-outer quadrant of right female breast': {'side':'R', 'quadrant':'LO'},
    'Malignant neoplasm of central portion of breast, female': {'side':'unspecified', 'quadrant':'central'},
    'Malignant neoplasm of lower-inner quadrant of female breast': {'side':'unspecified', 'quadrant':'LI'},
    'Malignant neoplasm of ovrlp sites of left female breast': {'side':'L', 'quadrant':'overlap'},
    'Malignant neoplasm of central portion of right female breast': {'side':'R', 'quadrant':'central'},
    'Malignant neoplasm of nipple and areola, right female breast': {'side':'R', 'quadrant':'nipple'},
    'Malignant neoplasm of axillary tail of left female breast': {'side':'L', 'quadrant':'axillary tail'},
    'Malignant neoplasm of upper-outer quadrant of female breast': {'side':'unspecified', 'quadrant':'UO'},
    'Malig neoplasm of upper-outer quadrant of unsp female breast': {'side':'unspecified', 'quadrant':'UO'},
    'Malig neoplasm of lower-outer quadrant of left female breast': {'side':'L', 'quadrant':'LO'},
    'Malignant neoplasm of central portion of unsp female breast': {'side':'unspecified', 'quadrant':'central'},
    'Malig neoplasm of lower-outer quadrant of unsp female breast': {'side':'unspecified', 'quadrant':'LO'},
    'Malignant neoplasm of axillary tail of right female breast': {'side':'R', 'quadrant':'axillary tail'},
    'Malignant neoplasm of ovrlp sites of unsp female breast': {'side':'unspecified', 'quadrant':'overlap'},
    'Malignant neoplasm of upper-outer quadrant of breast, female': {'side':'unspecified', 'quadrant':'UO'},
    'Malignant neoplasm of overlapping sites of breast, female': {'side':'unspecified', 'quadrant':'overlap'},
    'Malignant neoplasm of central portion of female breast': {'side':'unspecified', 'quadrant':'central'},
    'Malignant neoplasm of axillary tail of breast, female': {'side':'unspecified', 'quadrant':'axillary tail'},
    'Malignant neoplasm of breast of unspecified site, female': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malig neoplasm of upper-inner quadrant of unsp female breast': {'side':'unspecified', 'quadrant':'UI'},
    'Secondary malignant neoplasm of breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malignant neoplasm of lower-outer quadrant of breast, female': {'side':'unspecified', 'quadrant':'LO'},
    'Malignant neoplasm of axillary tail of unsp female breast': {'side':'unspecified', 'quadrant':'axillary tail'},
    'Malignant neoplasm of nipple and areola, unsp female breast': {'side':'unspecified', 'quadrant':'nipple'},
    'Malig neoplasm of lower-inner quadrant of unsp female breast': {'side':'unspecified', 'quadrant':'LI'},
    'Malignant neoplasm of axillary tail of female breast': {'side':'unspecified', 'quadrant':'axillary tail'},
    'Malignant neoplasm of unsp site of unspecified male breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malig neoplasm of lower-outer quadrant of right male breast': {'side':'R', 'quadrant':'LO'},
    'Malig neoplasm of upper-inner quadrant of left male breast': {'side':'L', 'quadrant':'UI'},
    'Malignant neoplasm of upper-inner quadrant of breast, female': {'side':'unspecified', 'quadrant':'UI'},
    'Malignant neoplasm of nipple and areola of female breast': {'side':'unspecified', 'quadrant':'nipple'},
    'Malignant neoplasm of nipple and areola, female': {'side':'unspecified', 'quadrant':'nipple'},
    'Malignant neoplasm of nipple and areola, right male breast': {'side':'R', 'quadrant':'nipple'},
    'Malignant neoplasm of central portion of right male breast': {'side':'R', 'quadrant':'central'},
    'Malignant neoplasm of lower-inner quadrant of breast, female': {'side':'unspecified', 'quadrant':'LI'},
    'Malig neoplasm of lower-outer quadrant of left male breast': {'side':'L', 'quadrant':'LO'},
    'Malig neoplasm of upper-outer quadrant of right male breast': {'side':'R', 'quadrant':'UO'},
    'Malig neoplasm of upper-inner quadrant of right male breast': {'side':'R', 'quadrant':'UI'},
    'Malignant neoplasm of other and unspecified sites of male breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malignant neoplasm of unspecified site of right male breast': {'side':'R', 'quadrant':'unspecified'},
    'Malig neoplasm of lower-inner quadrant of left male breast': {'side':'L', 'quadrant':'LI'},
    'Malig neoplasm of upper-outer quadrant of left male breast': {'side':'L', 'quadrant':'UO'},
    'Malignant neoplasm of breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malignant neoplasm of nipple and areola, left male breast': {'side':'L', 'quadrant':'nipple'},
    'Malignant neoplasm of axillary tail of right male breast': {'side':'R', 'quadrant':'axillary tail'},
}

In [36]:
df['side'] = df['breast_cancer_diagnosis_desc'].apply(lambda x: map_cancer_desc[x]['side'])
df['quadrant'] = df['breast_cancer_diagnosis_desc'].apply(lambda x: map_cancer_desc[x]['quadrant'])

In [41]:
df['side'].value_counts(dropna=False)

side
L              15031
R              14783
unspecified     9705
Name: count, dtype: int64

In [42]:
df['quadrant'].value_counts(dropna=False)

quadrant
unspecified      19146
UO                8419
overlap           3787
UI                2499
central           1741
LO                1700
LI                1125
nipple             781
axillary tail      321
Name: count, dtype: int64